In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as torchdata
import numpy as np
from tensorboardX import SummaryWriter
torch.manual_seed(1)

In [2]:
USE_CUDA = torch.cuda.is_available() # gpu 사용

### MNIST 데이터

In [3]:
BATCH_SIZE = 4

In [18]:
class CustomDataset(torchdata.Dataset):
    def __init__(self):
        
        data = np.loadtxt('./weight_decay_data.csv',
                        delimiter=',', dtype=np.float32)
        
        self.length = data.shape[0]
        self.x = torch.Tensor(data[:,:2]).float()
        self.y = torch.Tensor(data[:,2]).unsqueeze(1).float()
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
        
    def __len__(self):
        return self.length 

In [19]:
train_dataset = CustomDataset()

train_loader = torchdata.DataLoader(dataset=train_dataset,
                                                      batch_size=BATCH_SIZE, 
                                                      shuffle=True,
                                                      num_workers=2)

## Model 

In [20]:
class NN(nn.Module):
    def __init__(self,hidden_size):
        super(NN,self).__init__()
        self.l1 = nn.Linear(2,hidden_size)
        self.l2 = nn.Linear(hidden_size,1)
    
    def forward(self,inputs):
        outputs = F.relu(self.l1(inputs))
        return F.sigmoid(self.l2(outputs))

## Learning rate scheduling 

### 1. LambdaLR 

In [37]:
LR = 0.1
model = NN(6)
optimizer = optim.Adam(model.parameters(),lr=LR)
loss_function = nn.BCELoss()

In [38]:
lambda1 = lambda epoch: epoch // 10
lambda2 = lambda epoch: 0.95 ** epoch
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])
for epoch in range(100):
    
    for i,(inputs,targets) in enumerate(train_loader):
        model.zero_grad()
        pred = model(Variable(inputs))
        loss = loss_function(pred,Variable(targets).float())
        loss.backward()
        scheduler.step()
    
    if epoch % 30 ==0:
        print(loss.data[0],scheduler.get_lr())

0.8054615259170532 [0.1]
0.6733381152153015 [4.0]
0.7170065641403198 [7.9]
0.689446747303009 [11.8]
